** This code can be executed in google colab **

Content ready for pre-processing of the RNA Gquadruplexes bed file.

 The datasets can be downloaded from ' https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE77282 ' reference
 



In [ ]:
# Downloading GEO Datasets
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE77nnn/GSE77282/suppl/GSE77282_K_hits.bed.gz
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE77nnn/GSE77282/suppl/GSE77282_KPDS_hits.bed.gz
!wget https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh37_latest/refseq_identifiers/GRCh37_latest_genomic.fna.gz

--2022-05-18 13:56:50--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE77nnn/GSE77282/suppl/GSE77282_K_hits.bed.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138515 (135K) [application/x-gzip]
Saving to: ‘GSE77282_K_hits.bed.gz’

GSE77282_K_hits.bed 100%[===================>] 135.27K   248KB/s    in 0.5s    

2022-05-18 13:56:53 (248 KB/s) - ‘GSE77282_K_hits.bed.gz’ saved [138515/138515]

--2022-05-18 13:56:53--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE77nnn/GSE77282/suppl/GSE77282_KPDS_hits.bed.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 441833 (431K) [ap

In [ ]:
# install pybedtools
!apt-get install bedtools
!pip install pybedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bedtools is already the newest version (2.26.0+dfsg-5).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.


In [ ]:
# decompress files
!gunzip -rk GRCh37.p13.genome.fa.gz
!gunzip GSE77282_KPDS_hits.bed.gz
!gunzip GSE77282_K_hits.bed.gz


gzip: GRCh37.p13.genome.fa.gz: No such file or directory


In [ ]:
import pandas as pd

# Reading the file 
df1= pd.read_csv("GSE77282_K_hits.bed", sep='\t',header=None)
df2=pd.read_csv("GSE77282_KPDS_hits.bed",sep='\t', header=None)
K_hits= df1[[0,1,2,3,4,5]] 
KPDS_hits = df2[[0,1,2,3,4,5]] 

#Selecting the required cols and restoring to new files 

K_hits.to_csv("K_hits.csv", index=False, sep='\t',header=False)
KPDS_hits.to_csv("KPDS_hits.csv", index=False, sep='\t', header=False)


The provided datasets are of various length, we are here trying to unify the length of the reads upto 200 ntd. 


>  The sequences are extended if we have reads shorter than 200 ntd with the flanking transcripts



> If Sequences are longer than 200 nt, those read are centered and cut to 200 nt



>The sequence of the size 200 were consider same








In [ ]:

# Extending sequences shorter than 200 nucleotides

import pandas as pd
import glob, os
import numpy as np
import pandas as pd
import io
import sys
#os.chdir("/content/")

fixed_len=200
df = pd.read_csv('KPDS_hits.csv', sep='\t', comment='t', header=None)
df.columns = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand']
print(df)
chr_start=df.chromStart
chr_end=df.chromEnd
len_diff=chr_end-chr_start
ext_chrstr= chr_start -(fixed_len-len_diff)/2
ext_chrend= chr_end +(fixed_len-len_diff)/2
team=pd.DataFrame(list(zip(ext_chrstr,ext_chrend)))
team.columns=['ext_chromStart','ext_chromEnd'] 
df['chromStart'] = team['ext_chromStart']
df['chromEnd'] = team['ext_chromEnd']

df.to_csv('KPDS_out.bed', sep='\t', index=False, header=False)

      chrom  chromStart   chromEnd       name     score strand
0      chr1      762160     762190  LINC00115  0.295290      -
1      chr1      892334     892364      NOC2L  0.297222      -
2      chr1      892371     892401      NOC2L  0.488479      -
3      chr1      898579     898609     KLHL17  0.635253      +
4      chr1      900936     900966     KLHL17  0.674253      +
...     ...         ...        ...        ...       ...    ...
11362  chrX   154299805  154299835      BRCC3  0.308929      +
11363  chrX   154351100  154351130      BRCC3  0.853233      +
11364  chrX   154719703  154719733      TMLHE  0.944949      -
11365  chrX   154719710  154719740      TMLHE  0.914683      -
11366  chrX   154842552  154842582      TMLHE  0.678597      -

[11367 rows x 6 columns]


**Pipeline run on Command Line Interface for extracting fasta files from bed files**

---

setting up conda environment, make sure you are in the right path and have all files in the same directory






---


**Indexing** of the downloaded Human Reference genome using samtools



> samtools faidx GRCh37.p13.genome.fa



# **fetching fasta files from Bed file format**


> bedtools getfasta -s -fo KPDS_hits.fasta -name -fi GRCh37.p13.genome.fa -bed KPDS_hits.csv

> bedtools getfasta -s -fo K_hits.fasta -name -fi GRCh37.p13.genome.fa -bed K_hits.csv








In [ ]:
# Commented out IPython magic to ensure Python compatibility.
#All sequences which were longer than 200 nt were centered and cut to the length of 200 nt.
#Reads of 200 nts were kept as it is
# %pip install pysam
# %pip install biopython
from pysam import FastxFile
import random
import math
from Bio import SeqIO
from Bio.Seq import Seq


fasta_q_file = "KPDS_hits.fasta"
out_filename = "KPDS_hits_out.fasta"
size_trim = 200


with FastxFile(fasta_q_file) as fh, open(out_filename, mode='w') as fout:

  for entry in fh:
    sequence_id = entry.name
    sequence = str(entry.sequence)
    seq_length = len(sequence)
    if seq_length > size_trim:
      start_range=math.floor((seq_length-size_trim)/2)
      end_range=start_range+size_trim
      new_seq=sequence[math.floor(start_range):math.ceil(end_range)]
      fout.write(">{}\n{}\n".format(sequence_id, new_seq))
      
    else:
      if seq_length==size_trim :
        new_seq=sequence
        fout.write(">{}\n{}\n".format(sequence_id, sequence))

    continue

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
#All sequences which were longer than 200 nt were centered and cut to the length of 200 nt.
#Shorter sequences were randomly padded from both sides with Ns to become 200 nt long.
#Reads of 200 nts were kept as it is
# %pip install pysam
# %pip install biopython
from pysam import FastxFile
import random
from Bio import SeqIO
from Bio.Seq import Seq

#from pysam import FastxFile

#fasta_q_file = "K_hits.fasta"
#out_filename = "K_hits_padded.fasta"
size_trim = 200


with FastxFile(fasta_q_file) as fh, open(out_filename, mode='w') as fout:
    print(fh)
    for entry in fh:
        sequence_id = entry.name
        sequence = str(entry.sequence)
        seq_length = len(sequence)
        if seq_length > size_trim:
          x=(seq_length-size_trim)/2
          
          y=seq_length-(x-1)
          new_seq=sequence[int(x):int(y)]
          
         
          fout.write(">{}\n{}\n".format(sequence_id, new_seq))
        else:
          if seq_length < size_trim :
            x=(size_trim-seq_length)/2
            y= "N"*int(x)
            new_seq=y+sequence+y 
            
            fout.write(">{}\n{}\n".format(sequence_id, new_seq))
          else:
            if seq_length==trim_size :
              fout.write(">{}\n{}\n".format(sequence_id, sequence))

            
            continue

**Removing fasta header and the positive files are ready**

sed '/^>/d' K_hits_padded.fasta > K_hits_padded_header.fasta

sed '/^>/d' KPDS_hits_padded.fasta > KPDS_hits_padded_header.fasta

**Extracting Negative datasets**


We used bedtools shuffle for suffling the reads over transcripts which were not from the positive sets.


> The option -incl bed file contains all the transcripts with Exons,CDS,5UTR and 3UTR regions and the transcripts less than 300 bases were discarded.



> Those positive which were not considered were excluded from the input with -excl options




#bedtools suffle
bedtools shuffle -i  K_positive.bed -g GRCh37.p13.genome -incl GRCh37.p13_gtf-l300.bed -excl  K_positive.bed > K_shuffled_negative.bed
bedtools shuffle -i KPDS_positive.bed -g GRCh37.p13.genome -incl GRCh37.p13_gtf-l300.bed -excl K_new_negative.bed > KPDS_shuffled_negative.bed



